###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Educación Nacional

In [1]:
import pandas as pd

###Descarga del dataset desde el endpoint - archivo 3y4s-dmxy.csv

Contiene información estadística de primera infancia a relacionada con indicadores de (EDUCACIÓN INICIAL ICBF, NIÑOS Y NIÑAS EN PREESCOLAR CON EDUCACIÓN INICIAL y NIÑOS Y NIÑAS CON EDUCACIÓN INICIAL), con un histórico desde el 2015 hasta 2023 datos oficiales definitivos.


In [2]:
early_childhood_indicators_mineducacion = pd.read_csv("https://www.datos.gov.co/resource/3y4s-dmxy.csv?$limit=5000000", dtype={'cod_dane_mun': str})
early_childhood_indicators_mineducacion.head(3)

,fecha,fecha_corte,etc,cod_dane_depto,nombre_depto,cod_dane_mun,nombre_mun,indicador_1_educaci_n_inicial,indicador_2_ni_os_y_ni_as,indicador_3_total_educaci,indicador_4_concurrencia_de_atenciones
0,31-12-2015,31-12-2015,Amazonas,91,Amazonas,91263,El Encanto,113,0,113,2.1%
1,31-12-2015,31-12-2015,Amazonas,91,Amazonas,91405,La Chorrera,60,0,60,3.3%
2,31-12-2015,31-12-2015,Amazonas,91,Amazonas,91407,La Pedrera,43,0,43,2.3%


###Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
early_childhood_indicators_mineducacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10107 entries, 0 to 10106
Data columns (total 11 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   fecha                                   10107 non-null  object
 1   fecha_corte                             10107 non-null  object
 2   etc                                     10107 non-null  object
 3   cod_dane_depto                          10107 non-null  int64 
 4   nombre_depto                            10107 non-null  object
 5   cod_dane_mun                            10107 non-null  object
 6   nombre_mun                              10106 non-null  object
 7   indicador_1_educaci_n_inicial           10107 non-null  int64 
 8   indicador_2_ni_os_y_ni_as               10107 non-null  int64 
 9   indicador_3_total_educaci               10107 non-null  int64 
 10  indicador_4_concurrencia_de_atenciones  10107 non-null  object
dtypes:

- Descripcion de cada columna

In [4]:
column_mapping = {
    "fecha": "Es la fecha en la cual se registró el evento asociado al indicador.",
    "fecha_corte": "Es es la fecha en la cual se realiza la actualización de la información trimestral",
    "etc": "Nombre Entidad Territorial Certificada o Nombre Secretaría de Educación Certificada",
    "cod_dane_depto": "Código departamento Divipola",
    "nombre_depto": "Nombre del departamento",
    "cod_dane_mun": "Código municipio divipola",
    "nombre_mun": "Nombre del Municipio",
    "indicador_1_educaci_n_inicial": "Niños y niñas atendidos por el ICBF en servicios de educación inicial en el marco de la atención integral. Incluye Mujeres Gestantes.",
    "indicador_2_ni_os_y_ni_as": "Niños y niñas de preescolar con educación inicial en el marco de la atención integral",
    "indicador_3_total_educaci": "Total de niños y niñas atendidos por el ICBF (Incluyendo Mujeres Gestantes) y Niños y niñas de preescolar con educación inicial en el marco de la atención integral",
    "indicador_4_concurrencia_de_at": "Porcentaje de niños y niñas en servicios de educación inicial en el marco de la atención integral que cuentan con 6 o más atenciones priorizadas."

}

In [5]:
columns = early_childhood_indicators_mineducacion.columns

In [6]:
# Verificar el numero de valores únicos que se guarda en cada columna
for col in columns:
  print(f'{col}: {early_childhood_indicators_mineducacion[col].nunique()}')

fecha: 9
fecha_corte: 9
etc: 96
cod_dane_depto: 33
nombre_depto: 33
cod_dane_mun: 1123
nombre_mun: 1038
indicador_1_educaci_n_inicial: 2556
indicador_2_ni_os_y_ni_as: 841
indicador_3_total_educaci: 2712
indicador_4_concurrencia_de_atenciones: 1599


In [7]:
# Eliminar columnas que se considera innecesarias para el proyecto
delete_columns = ['fecha_corte','etc', 'cod_dane_depto']
# Eliminando las columnas especificadas
early_childhood_indicators_mineducacion = early_childhood_indicators_mineducacion.drop(delete_columns, axis=1)

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [8]:
# Obtener las columnas que no son numéricas
non_numeric_columns = ['nombre_depto', 'cod_dane_mun', 'nombre_mun', 'indicador_4_concurrencia_de_atenciones']
print(non_numeric_columns)

['nombre_depto', 'cod_dane_mun', 'nombre_mun', 'indicador_4_concurrencia_de_atenciones']


- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [9]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars =  [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [10]:
# Aplicar la función a todas las columnas categóricas
early_childhood_indicators_mineducacion[non_numeric_columns] = early_childhood_indicators_mineducacion[non_numeric_columns].astype(str)
for col in non_numeric_columns:
    early_childhood_indicators_mineducacion[col] = early_childhood_indicators_mineducacion[col].apply(remove_accents_and_special_chars)

In [11]:
# Imprimir categorías únicas para columnas de tipo object
for column in non_numeric_columns:
    print(f"Categorías en la columna '{column}':")
    print(early_childhood_indicators_mineducacion[column].unique())
    print()

Categorías en la columna 'nombre_depto':
['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'QUINDIO' 'ATLANTICO' 'SANTANDER'
 'BOGOTA D.C.' 'BOLIVAR' 'BOYACA' 'VALLE DEL CAUCA' 'CALDAS' 'CAQUETA'
 'CASANARE' 'CAUCA' 'CESAR' 'CUNDINAMARCA' 'CHOCO' 'MAGDALENA' 'CORDOBA'
 'NORTE DE SANTANDER' 'RISARALDA' 'GUAINIA' 'GUAVIARE' 'HUILA' 'TOLIMA'
 'NARINO' 'LA GUAJIRA' 'META' 'PUTUMAYO'
 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA' 'SUCRE'
 'VAUPES' 'VICHADA']

Categorías en la columna 'cod_dane_mun':
['91263' '91405' '91407' ... '85001' '76892' '25899']

Categorías en la columna 'nombre_mun':
['EL ENCANTO' 'LA CHORRERA' 'LA PEDRERA' ... 'YUMBO' 'ZIPAQUIRA' 'NAN']

Categorías en la columna 'indicador_4_concurrencia_de_atenciones':
['2.1' '3.3' '2.3' ... '77.68' '81.53' '70.65']



- Estandarizar algunas columnas categoricas

In [12]:
# Columna

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.


Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigomunicipioatencion"

In [13]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


- Longitud de los códigos municipios en el df

In [14]:
# Convertir a strings
early_childhood_indicators_mineducacion["cod_dane_mun"] = early_childhood_indicators_mineducacion["cod_dane_mun"].astype(str)

# Calcular la longitud de cada valor en la columna
longitud_mpio = early_childhood_indicators_mineducacion["cod_dane_mun"].apply(len)

# Longitudes de los códigos
print(f'Los códigos de los mpios tiene longitudes de :{longitud_mpio.unique()} caracteres')

Los códigos de los mpios tiene longitudes de :[5] caracteres


In [15]:
# Contar registros por longitud
long_df = longitud_mpio.value_counts().reset_index()
long_df.rename(columns={'cod_mpio': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitud_mpio))
long_df.head()

,cod_dane_mun,count,percentage
0,5,10107,1.0


In [16]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitud_mpio.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = early_childhood_indicators_mineducacion[longitud_mpio == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_dane_mun']])
    print()

Muestra de registros con longitud 5:
  cod_dane_mun
0        91263
1        91405
2        91407
3        91430
4        91001



Nota: Todos los códigos del df que se esta analizando tiene un string de 5 digitos igual al almacenado en la base de datos, asi que pasamos  a verificar si todos los códigos de este df corresponden a valores reales

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [17]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [18]:
# Comparar listas de códigos
compare_lists(early_childhood_indicators_mineducacion['cod_dane_mun'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en early_childhood_indicators_mineducacion", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en early_childhood_indicators_mineducacion que no están en Códigos de municipios dept_mpios_codes:
{'27086', '94663'}


- Analizar cod de mpio que no corresponde a un cod real

In [19]:
mpio_cod_ = early_childhood_indicators_mineducacion[early_childhood_indicators_mineducacion['cod_dane_mun'].isin(['27086', '94663'])]
mpio_cod_['nombre_mun'].value_counts()

,count
nombre_mun,
BELEN DE BAJIRA,9
MAPIRIPANA,9


Nota: BELEN DE BIJARA figura en la base de datos como un corregimiento del municipio de MUTATA y Mapiripana figura como corregimiento de Barrancominas del Guania

In [20]:
# Verificar cod municipios de MUTATA Y BARRANCOMINAS
dept_mpios_codes[dept_mpios_codes['mupio_name'].isin(['MUTATA','BARRANCOMINAS'])]

,dept_mpio_code,dept_name,mupio_name
77,05480,ANTIOQUIA,MUTATA
1093,94343,GUAINIA,BARRANCOMINAS


In [21]:
# Reemplazar el valor '27086' por '05480' en la columna 'codigomunicipioatencion'
early_childhood_indicators_mineducacion['cod_dane_mun'] = early_childhood_indicators_mineducacion['cod_dane_mun'].replace({'27086':'05480', '94663':'05480'})

- Verificar si cambio se aplicó

In [22]:
# Comparar listas de códigos
compare_lists(early_childhood_indicators_mineducacion['cod_dane_mun'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en early_childhood_indicators_mineducacion", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en early_childhood_indicators_mineducacion que no están en Códigos de municipios dept_mpios_codes:
set()


- Convertir columna fechainscripcionbeneficiario a tipo date

In [24]:
# Convertir la columna 'fecha' a tipo datetime, especificando que el día viene primero
early_childhood_indicators_mineducacion['fecha'] = pd.to_datetime(early_childhood_indicators_mineducacion['fecha'], dayfirst=True, errors='coerce')

# Extraer solo el año
early_childhood_indicators_mineducacion['fecha'] = early_childhood_indicators_mineducacion['fecha'].dt.year

In [25]:
early_childhood_indicators_mineducacion['fecha'].unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023], dtype=int32)

### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

- Eliminar columnas que dejaron de ser necesarias

In [26]:
# Eliminar columnas 'departamento', 'municipio' que ya no son necesarias
columns_to_drop = ['nombre_depto', 'nombre_mun']
early_childhood_indicators_mineducacion = early_childhood_indicators_mineducacion.drop(columns=columns_to_drop)

In [27]:
# Adicionar columna para trazabilidad de la fuente
# Lista de tuplas
early_childhood_indicators_mineducacion['source_id'] = 117

In [28]:
early_childhood_indicators_mineducacion.columns

Index(['fecha', 'cod_dane_mun', 'indicador_1_educaci_n_inicial',
       'indicador_2_ni_os_y_ni_as', 'indicador_3_total_educaci',
       'indicador_4_concurrencia_de_atenciones', 'source_id'],
      dtype='object')

In [29]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'fecha':'year',
    'cod_dane_mun': 'dane_code',
    'indicador_1_educaci_n_inicial':'ind_1_icbf-children',
    'indicador_2_ni_os_y_ni_as':'ind_2_prescholar_children',
    'indicador_3_total_educaci':'ind_3_total_children',
    'indicador_4_concurrencia_de_atenciones':'ind_4_overlaping_services',
    'source_id': 'source_id'
}

# Renombrar las columnas
early_childhood_indicators_mineducacion.rename(columns=translation_map, inplace=True)

In [30]:
#Estructura final del dataset a integrar a la base de datos
early_childhood_indicators_mineducacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10107 entries, 0 to 10106
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   year                       10107 non-null  int32 
 1   dane_code                  10107 non-null  object
 2   ind_1_icbf-children        10107 non-null  int64 
 3   ind_2_prescholar_children  10107 non-null  int64 
 4   ind_3_total_children       10107 non-null  int64 
 5   ind_4_overlaping_services  10107 non-null  object
 6   source_id                  10107 non-null  int64 
dtypes: int32(1), int64(4), object(2)
memory usage: 513.4+ KB


In [31]:
early_childhood_indicators_mineducacion.head()

,year,dane_code,ind_1_icbf-children,ind_2_prescholar_children,ind_3_total_children,ind_4_overlaping_services,source_id
0,2015,91263,113,0,113,2.1,117
1,2015,91405,60,0,60,3.3,117
2,2015,91407,43,0,43,2.3,117
3,2015,91430,0,0,0,0.0,117
4,2015,91001,3617,0,3617,30.0,117


## Salvar en archivo csv en el drive

In [32]:
# Guardar en archivos CSV en el drive
early_childhood_indicators_mineducacion.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/early_childhood_indicators_mineducacion.csv', index=False)